In [1]:
import requests
import pandas as pd 
from secrets_config import api_key # https://home.openweathermap.org/ 

In [2]:
# open weather api usage docs: https://openweathermap.org/current 
params = {
    "q": "perth",
    "units": "metric",
    "appid": api_key
} 
response = requests.get(f"http://api.openweathermap.org/data/2.5/weather", params=params)
print(response.status_code)
response_data = response.json()

200


In [3]:
# check the response 
print(response_data)

{'coord': {'lon': 115.8333, 'lat': -31.9333}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 18.95, 'feels_like': 18.02, 'temp_min': 17.23, 'temp_max': 20.61, 'pressure': 1016, 'humidity': 43}, 'visibility': 10000, 'wind': {'speed': 3.09, 'deg': 360}, 'clouds': {'all': 0}, 'dt': 1657274924, 'sys': {'type': 2, 'id': 63154, 'country': 'AU', 'sunrise': 1657235819, 'sunset': 1657272357}, 'timezone': 28800, 'id': 2063523, 'name': 'Perth', 'cod': 200}


In [4]:
# showing the data 

df_weather = pd.json_normalize(data=response_data)
df_weather

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1657274924,28800,2063523,Perth,200,115.8333,-31.9333,...,1016,43,3.09,360,0,2,63154,AU,1657235819,1657272357


In [5]:
# listing all columns 
df_weather.columns

Index(['weather', 'base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod',
       'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'wind.speed', 'wind.deg', 'clouds.all', 'sys.type', 'sys.id',
       'sys.country', 'sys.sunrise', 'sys.sunset'],
      dtype='object')

In [13]:
# selecting only necessary columns 

df_weather_selected = df_weather[["dt", "id", "name", "main.temp"]]
df_weather_selected

,dt,id,name,main.temp
0,1657274318,2063523,Perth,19.22


In [ ]:
# how to extract data from multiple cities? e.g. canberra, sydney, etc

# 1. create a list of cities (csv)
# 2. read list of cities (csv) 
# 3. request data for each city (json) and push to a list 
# 4. convert list into dataframe 

In [6]:
# read list of cities
df_cities = pd.read_csv("data/australian_capital_cities.csv")
df_cities.head()

,city_name
0,canberra
1,sydney
2,darwin
3,brisbane
4,adelaide


In [7]:
# request data for each city (json) and push to a list 
weather_data = []
for city_name in df_cities["city_name"]:
    params = {
        "q": city_name,
        "units": "metric",
        "appid": api_key
    }
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather", params=params)
    if response.status_code == 200: 
        weather_data.append(response.json())
    else: 
        raise Exception("Extracting weather api data failed. Please check if API limits have been reached.")

In [9]:
# show requested data 
print(weather_data)

[{'coord': {'lon': 149.1281, 'lat': -35.2835}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 5.91, 'feels_like': 1.79, 'temp_min': 4.69, 'temp_max': 6.88, 'pressure': 1015, 'humidity': 83}, 'visibility': 10000, 'wind': {'speed': 6.69, 'deg': 330}, 'clouds': {'all': 75}, 'dt': 1657275567, 'sys': {'type': 2, 'id': 2004200, 'country': 'AU', 'sunrise': 1657228317, 'sunset': 1657263876}, 'timezone': 36000, 'id': 2172517, 'name': 'Canberra', 'cod': 200}, {'coord': {'lon': 151.2073, 'lat': -33.8679}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 10.3, 'feels_like': 8.95, 'temp_min': 7.55, 'temp_max': 11.61, 'pressure': 1013, 'humidity': 60}, 'visibility': 10000, 'wind': {'speed': 8.23, 'deg': 270, 'gust': 13.38}, 'clouds': {'all': 0}, 'dt': 1657275421, 'sys': {'type': 2, 'id': 2002865, 'country': 'AU', 'sunrise': 1657227607, 'sunset': 1

In [11]:
# convert list into dataframe 
df_weather_cities = pd.json_normalize(weather_data)
df_weather_cities.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,wind.gust
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1657275567,36000,2172517,Canberra,200,149.1281,-35.2835,...,83,6.69,330,75,2,2004200,AU,1657228317,1657263876,NaN
1,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1657275421,36000,2147714,Sydney,200,151.2073,-33.8679,...,60,8.23,270,0,2,2002865,AU,1657227607,1657263588,13.38
2,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1657275641,34200,2073124,Darwin,200,130.8418,-12.4611,...,67,1.79,20,0,1,9574,AU,1657229925,1657271045,NaN
3,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1657275301,36000,2174003,Brisbane,200,153.0281,-27.4679,...,63,5.14,280,0,2,2005393,AU,1657226300,1657264020,NaN
4,"[{'id': 802, 'main': 'Clouds', 'description': ...",stations,10000,1657275308,34200,2078025,Adelaide,200,138.6000,-34.9333,...,69,4.63,210,40,2,2001763,AU,1657230790,1657266456,NaN
